In [1]:
using Pkg
Pkg.activate("/mnt/dv/wid/projects4/SolisLemus-network-merging/")
Pkg.instantiate()

using InPhyNet, PhyloNetworks
include("/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/simulation-scripts/helpers/helpers.jl")
cd("/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences")

  Activating project at `/mnt/dv/wid/projects4/SolisLemus-network-merging`


In [2]:
# "Instance" variables
ntaxa = 500
replicatenum = 1
ngt = 100
seq_len = 500
ils_level = "low"
m = 25
dmethod = "AGIC"

true_net = load_true_net_ils_adjusted_level1(ntaxa, replicatenum, ils_level)
# for e in true_net.edge
#     if e.length == -1. e.length = 0.473 end
# end

HybridNetwork, Rooted Network
1048 edges
1033 nodes: 501 tips, 16 hybrid nodes, 516 internal tree nodes.
tip labels: t3, t4, t22, t23, ...
(((((((t3:5.461,t4:5.461):8.785,(((((t22:0.619,t23:0.619):0.457,t18:1.076):3.687,t5:4.763):6.833,(((t13:1.347,(t17:1.198,(t19:1.035,t20:1.035):0.164):0.149):3.123,(t12:1.378)#H1:3.092::0.53):3.269,(((t8:2.042,t9:2.042):4.036,(t10:1.428,t11:1.428):4.649):0.225,(t6:2.448,(t16:1.283,((t24:0.29,t25:0.29):0.643,t21:0.934):0.35):1.164):3.855):1.437):3.857):1.451,((t2:7.424,(((t14:1.313,t15:1.313):0.065,#H1:0.0::0.47):1.034,t7:2.412):5.012):2.068,t1:9.492):3.555):1.199):5.029,(((((((t233:1.708,(t235:1.173,((t241:0.672,t242:0.672):0.003)#H9:0.498::0.554):0.535):0.496,(t243:0.454,t244:0.454):1.75):0.995,t229:3.199):3.167,(t227:4.628,(t228:4.245,(t230:2.571,t231:2.571):1.675):0.382):1.738):0.216,(((t236:0.886,t237:0.886):3.892)#H10:0.56::0.625,((((((((t238:0.736)#H11:0.0::0.548,t239:0.736):1.264,((t249:0.167,t250:0.167):1.515,t234:1.682):0.317):0.325,(t245:0.

In [3]:
# File paths
data_dir = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/temp_data/"
checkpoint_dir = joinpath(pwd(), "temp_data")
if !isdir(checkpoint_dir) mkdir(checkpoint_dir) end

truegt_file = joinpath(checkpoint_dir, "truegt_n$(ntaxa)_$(replicatenum)_$(ngt)_$(ils_level).treefile")
seq_file_prefix = joinpath(checkpoint_dir, "seqfile_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level).phy")
estgt_file = joinpath(checkpoint_dir, "estgt_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level).treefile")
net_file = joinpath(checkpoint_dir, "estnets_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod).netfile")

if !isfile(estgt_file) touch(estgt_file) end

# Estimated gene trees

Estimated gene trees are generated as follows:

```bash
#!/bin/bash

cd $inphynet/simulation-study/from-sequences/scripts

for ntaxa in 500
do
    for ils in low med high
    do
        for ngt in 100 1000
        do
            for seq_len in 500 1000
            do
                for replicate in $(seq 1 5)
                do
                    for m in 20 25
                    do
                        julia -p3 --project=../../.. ./simulate_estimated_gts.jl ${ntaxa} ${replicate} ${ngt} ${seq_len} ${ils} ${m}
                    done
                done
            done
        done
    done
done
```

In [4]:
# Consolidate all estimated gene trees into a single file
est_gts = Array{HybridNetwork}(undef, ngt)
for i = 1:ngt
    est_gts[i] = readTopology("$(estgt_file)_$(i)")
end

@everywhere GC.gc()

In [5]:
est_gts

100-element Vector{HybridNetwork}:
 HybridNetwork, Rooted Network
999 edges
1000 nodes: 501 tips, 0 hybrid nodes, 499 internal tree nodes.
tip labels: t498, t497, t487, t500, ...
(t498:0.036,t497:0.035,((t487:0.074,t500:0.073):0.017,(t499:0.076,(((((t485:0.11,t489:0.102):0.001,t486:0.104):0.085,t488:0.202):0.011,t481:0.182):0.153,(((t483:0.097,t484:0.115):0.525,((((((t476:0.296,(t477:0.256,t480:0.276):0.019):0.004,(((t490:0.069,(t493:0.065,t494:0.06):0.002):0.136,(t491:0.045,t492:0.038):0.155):0.002,t482:0.234):0.057):0.031,(t496:0.027,t495:0.025):0.242):0.045,(t479:0.231,t478:0.293):0.112):0.312,(((t471:0.013,t469:0.011):0.0,t470:0.01):0.333,((t453:0.076,((t472:0.023,t473:0.023):0.017,t464:0.053):0.033):0.228,((((((t459:0.043,(t462:0.043,t463:0.039):0.004):0.069,t452:0.127):0.002,t467:0.117):0.08,t468:0.166):0.061,(((t466:0.023,t465:0.026):0.077,(t460:0.038,t461:0.049):0.065):0.023,(t456:0.064,t455:0.06):0.053):0.141):0.013,t451:0.262):0.035):0.038):0.103):0.009,(((t454:0.079,t458:0.0

# Subset decomposition

In [5]:
_, _, nj_tre = estimate_nj_tree(est_gts)
subsets = sateIdecomp(nj_tre, m)

[NJ] Calculating AGIC.
[NJ] Estimating NJ tree.


21-element Vector{Vector{String}}:
 ["t101", "t102", "t105", "t107", "t108", "t113", "t114", "t109", "t111", "t112"  …  "t117", "t124", "t125", "t118", "t122", "t123", "t119", "t115", "t120", "t121"]
 ["t126", "t127", "t130", "t141", "t142", "t133", "t140", "t149", "t150", "t138"  …  "t135", "t147", "t148", "t132", "t143", "t144", "t128", "t129", "t136", "t137"]
 ["t276", "t278", "t279", "t280", "t282", "t289", "t291", "t292", "t293", "t294"  …  "t286", "t287", "t288", "t295", "t296", "t290", "t297", "t298", "t299", "t300"]
 ["t451", "t452", "t459", "t462", "t463", "t467", "t468", "t453", "t464", "t472"  …  "t465", "t466", "t469", "t471", "t470", "t454", "t457", "t458", "t474", "t475"]
 ["t351", "t354", "t368", "t370", "t369", "t371", "t372", "t373", "t352", "t353"  …  "t365", "t359", "t360", "t361", "t374", "t362", "t363", "t366", "t367", "t375"]
 ["t226", "t234", "t249", "t250", "t238", "t239", "t245", "t246", "t247", "t248"  …  "t231", "t229", "t233", "t235", "t243", "t244", "t241",

# Generate Condor data

In [ ]:
df_dir = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/CFs/"
nj_dir = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/NJs/"
nruns = 10
GC.gc()

for (i, subset_taxa) in enumerate(subsets)
    output_file = "$(net_file)_$(i)"
    runtime_file = "$(output_file).runtime"
    output_net_file = "$(output_file).netfile"
    
    if isfile(output_net_file) && isfile(runtime_file)
        log("SNaQ $(i)", "Already inferred.")
        continue
    end

    temp_gts = Array{HybridNetwork}(undef, length(est_gts))
    for i = 1:length(est_gts)
        temp_gts[i] = pruneTruthFromDecomp(est_gts[i], subset_taxa)
    end

    # 1. Quartets
    q, t = countquartetsintrees(temp_gts)
    df = silently() do
        readTableCF(writeTableCF(q, t))
    end
    CSV.write(joinpath(df_dir, "df_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod)_sub$(i).csv"), writeTableCF(df))

    # 2. Starting trees
    init_tree = pruneTruthFromDecomp(nj_tre, subset_taxa)
    tre_out = joinpath(nj_dir, "nj_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod)_sub$(i).tre")
    open(tre_out, "w+") do f
        write(f, writeTopology(init_tree))
    end

    # 3. Write info to condor input table
    tab_file = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/condor/inputs.tab"
    open(tab_file, "a") do f
        write(f, "$(ntaxa),$(replicatenum),$(ngt),$(seq_len),$(ils_level),$(m),$(i)\n")
    end

    # 4. Constraints inferred in Condor
end
@everywhere GC.gc()

# Infer constraints

This is done in Condor with the submit file `simulation-study/condor/submit.submit`

# InPhyNet

In [6]:
# Load estimated data
# est_constraints = Array{HybridNetwork}(undef, length(subsets))
est_constraints = Vector{HybridNetwork}([])
for i = 1:length(subsets)
    net_file = joinpath(
        "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/snaq_nets/",
        "snaq_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod)_sub$(i).out"
    )
    if !isfile(net_file)
        continue
    end

    net = split(readlines(net_file)[1], " -Ploglik")[1]
    push!(est_constraints, readTopology(net))
    # est_constraints[i] = readTopology(net)
end
println("$(length(est_constraints))/$(length(subsets)) constraints")

# est_constraint_runtimes = Vector{Float64}(
#     [parse(Float64, readlines("$(net_file)_$(i).runtime")[1]) for i=1:length(subsets)]
# )

7/21 constraints


In [8]:
est_constraints

7-element Vector{HybridNetwork}:
 HybridNetwork, Rooted Network
47 edges
48 nodes: 25 tips, 0 hybrid nodes, 23 internal tree nodes.
tip labels: t274, t275, t267, t261, ...
(t274,t275,(t267,(t261,(t251,((t257,t258):9.047,(((t252,((t268,t269):0.564,(t270,t271):1.234):9.827):2.864,((((t263,t262):1.051,t255):0.048,t256):0.311,(t259,(t266,(t272,t273):0.496):0.801):1.204):9.984):1.934,(t253,(t254,((t264,t265):0.528,t260):1.998):0.378):9.961):2.082):10.0):8.902):0.844):1.341);

 HybridNetwork, Rooted Network
47 edges
48 nodes: 25 tips, 0 hybrid nodes, 23 internal tree nodes.
tip labels: t441, t442, t428, t434, ...
(t441,t442,(t428,((t434,t435):3.9,((t449,t450):9.945,(((t429,(t430,t431):0.51):0.901,((t436,(t439,t440):0.419):2.315,((t437,t438):1.475,(t445,t446):1.542):1.1):0.259):2.05,(((t432,t433):10.0,(t443,t444):10.0):0.232,((t447,t448):9.966,(t427,t426):2.384):3.085):0.211):0.605):0.439):1.933):1.958);

 HybridNetwork, Rooted Network
47 edges
48 nodes: 25 tips, 0 hybrid nodes, 23 internal t

In [9]:
est_gts = Array{HybridNetwork}(undef, ngt)
for i = 1:ngt
    est_gts[i] = readTopology("$(estgt_file)_$(i)")
end
est_D, est_namelist = calculateAGIC(est_gts)
est_namelist = Vector{String}(est_namelist)
1

1

In [10]:
for c in est_constraints
    rootatnode!(c, getchildren(c.node[c.root])[1])
end

In [11]:
inphynet_time = @elapsed mnet = netnj(est_D, est_constraints, est_namelist, force_unrooted = true)

rootatnode!(mnet, "OUTGROUP")
rootatnode!(true_net, "OUTGROUP")
hardwiredClusterDistance(mnet, true_net, true), hardwiredClusterDistance(majorTree(mnet), majorTree(true_net), true)

# 7/21 constraints: (192, 132)
# not bad!!!

(192, 132)

In [28]:
for e in true_net.edge e.length = -1. end
println(writeTopology(majorTree(true_net)))

(((((((t3,t4),(((((t22,t23),t18),t5),(((t13,(t17,(t19,t20))),t12),(((t8,t9),(t10,t11)),(t6,(t16,((t24,t25),t21)))))),((t2,(t7,(t14,t15))),t1))),(((((((t233,(t235,(t241,t242))),(t243,t244)),t229),(t227,(t228,(t230,t231)))),((t226,((t247,t248),(((t239,t238),((t249,t250),t234)),(t245,t246)))),(t236,t237))),(t232,t240)),(((t257,t258),(((t267,(t274,t275)),t261),t251)),((t253,(((t264,t265),t260),t254)),((((t266,(t272,t273)),t259),(t255,((t262,t263),t256))),(t252,((t268,t269),(t270,t271)))))))),((t76,(((((t92,t93),t85),((((t90,t91),t82),(t83,t84)),t95)),(t78,t79)),((((((t99,t100),t88),(t86,t87)),(t97,t98)),t77),(((t94,t96),t89),(t80,t81))))),(((((t160,t161),(t167,t168)),((t162,t163),(t151,t155))),((t158,t159),((((t153,t154),(t156,t157)),(t165,t166)),(((((t174,t175),((t170,t171),t169)),t164),(t172,t173)),t152)))),(((((((t195,t196),(t191,t192)),(t180,t181)),((((t187,t188),t184),t183),t176)),(t177,(((t199,t200),(t185,(t189,t190))),((t179,t186),((t197,t198),t182))))),(t178,(t193,t194))),((((t401,

In [ ]:
save_estimated_gts_results(
    "$(ntaxa)", true_net, replicatenum, ngt, ils_level,
    m, dmethod, seq_len, mnet, est_constraints,
    est_gts, est_constraint_runtimes, inphynet_time
)